### FactRuEval nmt evaluation

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
import sys

sys.path.append("../")

warnings.filterwarnings("ignore")

In [2]:
import os


data_path = "/home/lis/ner/ulmfit/data/factrueval/"
train_path = os.path.join(data_path, "train_with_pos.csv")
valid_path = os.path.join(data_path, "valid_with_pos.csv")
model_dir = "/datadrive/elmo/"
config_name = "cnn_50_100_512_4096_sample.json"

In [3]:
import torch
torch.cuda.set_device(1)
torch.cuda.is_available(), torch.cuda.current_device()

(True, 1)

### 1. Create dataloaders

In [4]:
from modules.data.elmo_data import ElmoNerData as NerData

INFO:summarizer.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English


In [5]:
data = NerData.create(train_path, valid_path, model_dir, config_name)

For factrueval we use the following sample of labels:

In [6]:
print(data.label2idx)

{'<pad>': 0, '<bos>': 1, '<eos>': 2, 'O': 3, 'B_ORG': 4, 'I_ORG': 5, 'B_LOC': 6, 'B_PER': 7, 'I_PER': 8, 'I_LOC': 9}


### 2. Create model
For creating pytorch model we need to create `NerModel` object.

In [11]:
from modules.models.elmo_models import ElmoBiLSTMAttnNMT

In [21]:
model = ElmoBiLSTMAttnNMT.create(len(data.label2idx), model_dir, config_name,
                                 enc_hidden_dim=128, dec_hidden_dim=128, dec_embedding_dim=16)

INFO:root:char embedding size: 3896
INFO:root:word embedding size: 329681


In [22]:
model.decoder

NMTDecoder(
  (embedding): Embedding(10, 16)
  (lstm): LSTM(272, 128, batch_first=True)
  (attn): Linear(in_features=128, out_features=128, bias=True)
  (slot_out): Linear(in_features=256, out_features=10, bias=True)
  (loss): CrossEntropyLoss()
)

In [23]:
model.get_n_trainable_params()

783146

### 3. Create learner

For training our pytorch model we need to create `NerLearner` object.

In [24]:
from modules import NerLearner

In [25]:
learner = NerLearner(model, data,
                     best_model_path="/datadrive/models/factrueval/elmo_attn_nmt.cpt",
                     base_lr=0.001, lr_max=0.005, clip=5.0, use_lr_scheduler=False, sup_labels=data.id2label[4:])

INFO:root:Don't use lr scheduler...


### 4. Learn your NER model
Call `learner.fit`

In [ ]:
learner.fit(100, target_metric='prec')

### 5. Evaluate
Create new data loader from existing path.

In [34]:
from modules.data.elmo_data import get_elmo_data_loader_for_predict

In [35]:
dl = get_elmo_data_loader_for_predict(data_path + "valid_with_pos.csv", learner)

In [36]:
learner.load_model()

In [37]:
preds = learner.predict(dl)

IOB precision

In [28]:
from modules.train.train import validate_step
print(validate_step(learner.data.valid_dl, learner.model, learner.data.id2label, learner.sup_labels))

              precision    recall  f1-score   support

       B_ORG      0.823     0.769     0.795       260
       I_ORG      0.806     0.601     0.688       283
       B_LOC      0.898     0.862     0.880       195
       B_PER      0.932     0.932     0.932       191
       I_PER      0.952     0.915     0.933       130
       I_LOC      1.000     0.143     0.250        35

   micro avg      0.873     0.768     0.817      1094
   macro avg      0.902     0.704     0.746      1094
weighted avg      0.872     0.768     0.805      1094



Span precision

In [92]:
from modules.utils.plot_metrics import get_elmo_span_report

In [93]:
clf_report = get_elmo_span_report(dl, preds)
print(clf_report)

              precision    recall  f1-score   support

         PER      0.782     0.791     0.786       191
         LOC      0.793     0.764     0.778       195
         ORG      0.682     0.700     0.691       260

   micro avg      0.744     0.746     0.745       646
   macro avg      0.752     0.752     0.752       646
weighted avg      0.745     0.746     0.745       646

